<a href="https://colab.research.google.com/github/pradyot-09/DL-Reproducibility-Project/blob/master/CNNModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Import pytorch basic functions/classes
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Import torchvision functions/classes for MNIST import and data loaders
import torchvision
import torchvision.transforms as transforms

# Set device on which code is run
device = 'cuda'

In [0]:
# Define transform from PIL image to tensor and normalize to 1x768 pixels
transform = transforms.Compose([
  transforms.Resize(28),
  transforms.CenterCrop(28),
  transforms.ToTensor(),
  transforms.Normalize((0.5, ), (0.5, ))
])

# Set batch size for data loaders
batch_size = 32

# (Down)load training set
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

# (Down)load test set
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

In [0]:
# Define CNN model and its layers
class CNNModel(nn.Module):

    def __init__(self, hidden_size=128, dropout=0.25, hidden_dropout=0.5):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32,64, kernel_size=5)
        self.dropout1 = nn.Dropout2d(dropout)
        self.dropout2 = nn.Dropout2d(hidden_dropout)
        self.fc1 = nn.Linear(6400, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        return self.fc2(x)

In [34]:
# Setup model and move it to the GPU
net = CNNModel()
net.to(device)

# Set up loss function and optimizer: 
#     using cross entropy loss because it's better for classification task

criterion = torch.nn.CrossEntropyLoss()
#optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9)
optimizer = torch.optim.Adam(net.parameters(), lr=0.0001, weight_decay=0.001)

# Run over 100 epochs (1 epoch = visited all items in dataset)
for epoch in range(100): #200 for SGD reaches 0.002 loss

    running_loss = 0.0
    total = 0
    for i, (inputs, labels) in enumerate(trainloader, 0):
        

        # get the inputs; data is a list of [inputs, labels]
        inputs = inputs.to(device)
        target = labels.to(device).long()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        total += len(data)

        # print statistics
        running_loss += loss.item()
    # print every epoch
    print('[%d] loss: %.3f' % (epoch + 1, running_loss / total))

print('Finished Training')

# Save model after having finished training
PATH = './mnist_dropout_100_epoch.pth'
torch.save(net.state_dict(), PATH)

print('Saved Model')

[1] loss: 0.172
[2] loss: 0.062
[3] loss: 0.046
[4] loss: 0.040
[5] loss: 0.035
[6] loss: 0.032
[7] loss: 0.029
[8] loss: 0.028
[9] loss: 0.026
[10] loss: 0.025
[11] loss: 0.023
[12] loss: 0.023
[13] loss: 0.021
[14] loss: 0.022
[15] loss: 0.021
[16] loss: 0.020
[17] loss: 0.020
[18] loss: 0.020
[19] loss: 0.019
[20] loss: 0.018
[21] loss: 0.018
[22] loss: 0.017
[23] loss: 0.018
[24] loss: 0.017
[25] loss: 0.018
[26] loss: 0.016
[27] loss: 0.017
[28] loss: 0.016
[29] loss: 0.017
[30] loss: 0.016
[31] loss: 0.016
[32] loss: 0.016
[33] loss: 0.016
[34] loss: 0.015
[35] loss: 0.015
[36] loss: 0.015
[37] loss: 0.015
[38] loss: 0.015
[39] loss: 0.016
[40] loss: 0.015
[41] loss: 0.015
[42] loss: 0.014
[43] loss: 0.015
[44] loss: 0.014
[45] loss: 0.014
[46] loss: 0.013
[47] loss: 0.015
[48] loss: 0.014
[49] loss: 0.015
[50] loss: 0.014
[51] loss: 0.014
[52] loss: 0.014
[53] loss: 0.014
[54] loss: 0.015
[55] loss: 0.014
[56] loss: 0.014
[57] loss: 0.014
[58] loss: 0.014
[59] loss: 0.014
[60] l

In [0]:
# Instantiate model and load saved network parameters
net = CNNModel().to(device)
net.load_state_dict(torch.load(PATH))

# Run model on test set and determine accuracy
correct = 0
total = 0
with torch.no_grad():
    for (inputs, labels)) in testloader:
        inputs = inputs.to(device)
        target = labels.to(device)
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        _, target = torch.max(target.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

# Output model accuracy to user
print('Accuracy of the network on the 10000 test images: %d %% (%d wrong out of %d)' % (
    100 * correct / total, total - correct, total))